In [1]:
import sys
import os
from importlib import reload

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import re

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
from collections import Counter

In [6]:
try:
    # Путь для локальной машины (определяем корень проекта)
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    src_path = os.path.join(project_root, "src")

    # Проверяем, существует ли путь (т.е. файл "src" должен быть в корне проекта)
    if os.path.exists(src_path):
        sys.path.append(src_path)
        print("Local machine: Путь к 'src' добавлен.")
    else:
        raise FileNotFoundError(f"Path {src_path} not found on local machine.")

except:
    try:
        # Путь для Kaggle
        kaggle_src_path = '/kaggle/input/src'
        if os.path.exists(kaggle_src_path):
            sys.path.append(kaggle_src_path)
            print("Kaggle: Путь к 'src' добавлен.")
        else:
            raise FileNotFoundError(f"Path {kaggle_src_path} not found on Kaggle.")

    except Exception as e:
        print(f"Не удалось добавить путь к 'src'. Ошибка: {e}")

Local machine: Путь к 'src' добавлен.


In [7]:
import promiss
import langs_clean

In [8]:
try:
    # Путь для локальной машины
    file_path = r'C:\0\smf\train.csv' 
    dtr = pd.read_csv(file_path, lineterminator='\n')
    print('Train data loaded from local machine')

except:
    try:
        # Путь для Kaggle
        file_path = '/kaggle/input/animation-reels-classification/train.csv'
        dtr = pd.read_csv(file_path, lineterminator='\n')
        print('Train data loaded from Kaggle')

    except Exception as e:
        print(f'Train data was not loaded. Error: {e}')

Train data loaded from local machine


In [9]:
try:
    # Путь для локальной машины
    file_path = r'C:\0\smf\test.csv' 
    dtr = pd.read_csv(file_path, lineterminator='\n')
    print('Test data loaded from local machine')

except:
    try:
        # Путь для Kaggle
        file_path = '/kaggle/input/animation-reels-classification/test.csv'
        dtr = pd.read_csv(file_path, lineterminator='\n')
        print('Test data loaded from Kaggle')

    except Exception as e:
        print(f'Test data was not loaded. Error: {e}')

Test data loaded from local machine


In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
pd.set_option('display.max_rows', None)

In [12]:
dtr[:1]

,date,reel_name,yt_reel_id,url,text,seconds,is_shorts,broadcast,yt_channel_id,yt_channel_name,yt_ch_url,yt_channel_type,flag_closed,international,language
0,2020-10-26,пять маленьких монстров хэллоуин рифмы потешки Kids Tv Russia детские песни,RBw4j0aMWyM,https://www.youtube.com/watch?v=RBw4j0aMWyM,"[пять маленьких монстров хэллоуин рифмы потешки Kids Tv Russia детские песни ] привет дети, Kids Tv Russia приносит вам новую песню , учиться и веселиться.смотрите нашу коллекцию из лучших детских стишков и песни для детей с лучшей анимацией и мультфильмами на русском.Visit our website http://www.uspstudios.co/ for more Childrens Nursery Rhymes Kids VideosMusic and Lyrics: Copyright USP StudiosVideo: Copyright USP StudiosKIDS FIRST Kids Videos Nursery Rhymes Free App Download: http://m.onelink.me/1e8f6c16Bob the Train Plush Toys now available on Amazon. Buy yours today Bob the Train Amazon Store: https://amzn.to/2PCeSDS",960.0,0.0,none,UCfeljpDR__qqp-lLBJdiQkw,Kids Tv Russia песенки для дете,https://youtube.com/channel/UCfeljpDR__qqp-lLBJdiQkw,Детские,0.0,0.0,NaN


In [13]:
pd.reset_option('display.max_rows')

In [14]:
print('broadcast:       ',dtr.broadcast.unique())
print('yt_channel_type: ',dtr.yt_channel_type.unique())
print('flag_closed:     ',dtr.flag_closed.unique())
print('international:   ',dtr.international.unique())
print('is_shorts:       ',dtr.is_shorts.unique())

broadcast:        ['none' 'live' nan 'upcoming']
yt_channel_type:  ['Детские' 'Блогеры' 'Мультфильмы' nan 'Shorts']
flag_closed:      [ 0.  1. nan]
international:    [ 0.  1. nan]
is_shorts:        [ 0. nan  1.]


In [15]:
pivot_shorts_seconds = dtr.pivot_table(
    values='seconds',
    index='is_shorts',
    aggfunc=['min', 'mean', 'max']
)
pivot_shorts_seconds.columns = ['min_seconds', 'mean_seconds', 'max_seconds']
pivot_shorts_seconds = pivot_shorts_seconds.reset_index()
pivot_shorts_seconds

,is_shorts,min_seconds,mean_seconds,max_seconds
0,0.0,0.0,1948.002221,605861.0
1,1.0,4.0,26.553804,60.0


In [16]:
dtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55608 entries, 0 to 55607
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             55608 non-null  object 
 1   reel_name        55605 non-null  object 
 2   yt_reel_id       55608 non-null  object 
 3   url              55608 non-null  object 
 4   text             55608 non-null  object 
 5   seconds          49807 non-null  float64
 6   is_shorts        49807 non-null  float64
 7   broadcast        49807 non-null  object 
 8   yt_channel_id    55584 non-null  object 
 9   yt_channel_name  55580 non-null  object 
 10  yt_ch_url        55580 non-null  object 
 11  yt_channel_type  55580 non-null  object 
 12  flag_closed      55580 non-null  float64
 13  international    55580 non-null  float64
 14  language         337 non-null    object 
dtypes: float64(4), object(11)
memory usage: 6.4+ MB


In [17]:
dts.info()

NameError: name 'dts' is not defined

In [ ]:
dtr.yt_reel_id.nunique()

In [ ]:
dtr.cartoon.nunique()

In [ ]:
dtr.cartoon.value_counts(normalize=True)[:5]

In [ ]:
dtr.duplicated().sum()

Дубликатов нет

## Пропуски

In [ ]:
dtr[dtr.reel_name.isnull()]

In [ ]:
dts[dts.reel_name.isnull()]

Пропуски в поле `reel_name` - в малом количестве и не принадлежат к размеченным образцам. В базовом варианте обработки - удалим их

# Определим языки

In [ ]:
langs_clean.lang_distrib_train

# Анализ классов (выявление дисбаланса)

In [ ]:
# Преобразование категориальных данных
label_encoder = LabelEncoder()
dtr['cartoon_encoded'] = label_encoder.fit_transform(dtr['cartoon'])